In [3]:
import os
import pandas as pd
import numpy as np
import pandas_gbq
import sqlite3
import zipfile
import csv
import io
import glob

# Do our imports for the code
from google.cloud import bigquery
from google.oauth2 import service_account
from zipfile import ZipFile
from google.cloud.exceptions import NotFound

In [4]:
zip_folder = r'C:\Users\hills\Documents\Fall2023\ADA\wedge-project\data\WedgeZipOfZips_Big'
output_folder = 'delimeted_wedge_files'

In [8]:
if not os.path.exists(output_folder):
      os.makedirs(output_folder)

for zip_file_name in os.listdir(zip_folder):
    if zip_file_name.endswith('.zip'):
         zip_file_path = os.path.join(zip_folder, zip_file_name)

         output_zip_file_path = os.path.join(output_folder, zip_file_name)
         with ZipFile(output_zip_file_path, 'w') as output_zip:

            with ZipFile(zip_file_path, 'r') as zf:
                for file_name in zf.namelist():
                    if file_name.endswith('.csv'):
                        csv_file_name = file_name
                        with zf.open(csv_file_name, 'r') as input_file:
                            input_file = io.TextIOWrapper(input_file, encoding="utf-8")

                            output_file_name = os.path.splitext(csv_file_name)[0] + '_delimited.csv'
                            #print(output_zip_file_path)
                            
                            output_zip.writestr(output_file_name, '\n'.join(','.join(line.split(';')) for line in input_file))
                            #above will only work on ; delimted files
                

print("csv files have been proceessed and saved back to their respective zip files")

csv files have been proceessed and saved back to their respective zip files


In [4]:
#zip-files = os.listdir("WedgeZipOfZips")
#zip_folder = "no_headers" - was not commented out in source code
#zip_files = os.listdir("text")

zip_files = [file for file in os.listdir(zip_folder) if file.endswith(".zip")]

In [6]:
os.getcwd()

'c:\\Users\\hills\\Documents\\Fall2023\\ADA\\wedge-project'

In [5]:
column_types = {
    "datetime": "TIMESTAMP",  # 1
    "register_no": "FLOAT",   # 2
    "emp_no": "FLOAT",        # 3
    "trans_no": "FLOAT",      # 4
    "upc": "STRING",          # 5
    "description": "STRING",  # 6
    "trans_type": "STRING",   # 7
    "trans_subtype": "STRING", # 8
    "trans_status": "STRING",  # 9
    "department": "FLOAT",     # 10
    "quantity": "FLOAT",       # 11
    "Scale": "FLOAT",          # 12
    "cost": "FLOAT",           # 13
    "unitPrice": "FLOAT",      # 14
    "total": "FLOAT",          # 15
    "regPrice": "FLOAT",       # 16
    "altPrice": "FLOAT",       # 17
    "tax": "FLOAT",            # 18
    "taxexempt": "FLOAT",      # 19
    "foodstamp": "FLOAT",      # 20
    "wicable": "FLOAT",        # 21
    "discount": "FLOAT",       # 22
    "memDiscount": "FLOAT",    # 23
    "discountable": "FLOAT",   # 24
    "discounttype": "FLOAT",   # 25
    "voided": "FLOAT",         # 26
    "percentDiscount": "FLOAT",# 27
    "ItemQtty": "FLOAT",       # 28
    "volDiscType": "FLOAT",    # 29
    "volume": "FLOAT",         # 30
    "VolSpecial": "FLOAT",     # 31
    "mixMatch": "FLOAT",       # 32
    "matched": "FLOAT",        # 33
    "memType": "BOOLEAN",      # 34
    "staff": "BOOLEAN",        # 35
    "numflag": "FLOAT",        # 36
    "itemstatus": "FLOAT",     # 37
    "tenderstatus": "FLOAT",   # 38
    "charflag": "STRING",      # 39
    "varflag": "FLOAT",        # 40
    "batchHeaderID": "BOOLEAN", # 41
    "local": "FLOAT",          # 42
    "organic": "FLOAT",        # 43
    "display": "BOOLEAN",      # 44
    "receipt": "FLOAT",        # 45
    "card_no": "FLOAT",        # 46
    "store": "FLOAT",          # 47
    "branch": "FLOAT",         # 48
    "match_id": "FLOAT",       # 49
    "trans_id": "FLOAT"        # 50
}

In [28]:
output_folder = "with_headers"
os.makedirs(output_folder, exist_ok = True)

headers = dict()

for this_zf in zip_files:
    with ZipFile(os.path.join(r'C:\Users\hills\Documents\Fall2023\ADA\wedge-project\data\WedgeZipOfZips_Big', this_zf), 'r') as zf:
        zipped_files = zf.namelist()

        for file_name in zipped_files:
            input_file = zf.open(file_name, 'r')
            input_file = io.TextIOWrapper(input_file, encoding="utf-8")

            first_line = input_file.readline()

            is_header = any(keyword in first_line for keyword in column_types.keys())

            headers[file_name] = is_header

            if not is_header:
                header_row = ','.join(column_types.keys()) + '\n'
                input_file.seek(0)
                file_contents = input_file.read()
                input_file.close()

                subfolder_path = os.path.join(output_folder, os.path.splitext(file_name)[0])
                os.makedirs(subfolder_path, exist_ok=True)


                output_file_path = os.path.join(subfolder_path, file_name)
                with open(output_file_path, 'w', encoding="utf-8") as output_file:
                    output_file.write(header_row + file_contents)

                print(f"file: [file_name], has header: [is_header])")

for foldername, subfolders, filenames in os.walk(output_folder):
    for subfolder in subfolders:
        subfolder_path = os.path.join(output_folder, subfolder)
        zip_file_path = os.path.join(output_folder, subfolder + ".zip")
        with ZipFile(zip_file_path, 'w') as new_zip:
            for root, dirs, files in os.walk(subfolder_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    new_zip.write(file_path, os.path.relpath(file_path, subfolder_path))


for subfolder in os.listdir(output_folder):
    subfolder_path = os.path.join(output_folder, subfolder)
    if os.path.isdir(subfolder_path):
        for file in os.listdir(subfolder_path):
            file_path = os.path.join(subfolder_path, file)
            os.remove(file_path)
        os.rmdir(subfolder_path)

file: [file_name], has header: [is_header])
file: [file_name], has header: [is_header])
file: [file_name], has header: [is_header])
file: [file_name], has header: [is_header])
file: [file_name], has header: [is_header])
file: [file_name], has header: [is_header])
file: [file_name], has header: [is_header])
file: [file_name], has header: [is_header])
file: [file_name], has header: [is_header])
file: [file_name], has header: [is_header])
file: [file_name], has header: [is_header])
file: [file_name], has header: [is_header])
file: [file_name], has header: [is_header])
file: [file_name], has header: [is_header])
file: [file_name], has header: [is_header])


In [7]:
service_path = r"C:\Users\hills\Documents\Fall2023\ADA\wedge-project\leafy-sunrise-403222-f51fcd80b921.json"
service_file = 'leafy-sunrise-403222-f51fcd80b921.json' # change this to your authentication information  

private_key =service_path + service_file
credentials = service_account.Credentials.from_service_account_file(r"C:\Users\hills\Documents\Fall2023\ADA\wedge-project\leafy-sunrise-403222-f51fcd80b921.json")

gbq_proj_id = "leafy-sunrise-403222" # change this to your project. 
gbq_dataset_id = "wedge_data"

# And create a client to talk to GBQ
client = bigquery.Client(credentials=credentials, project=gbq_proj_id)

zip_dir = 'wedge_zips'

In [8]:
schema = [
    {"name": "datetime", "type": "TIMESTAMP"},     # 1
    {"name": "register_no", "type": "FLOAT"},      # 2
    {"name": "emp_no", "type": "FLOAT"},           # 3
    {"name": "trans_no", "type": "FLOAT"},         # 4
    {"name": "upc", "type": "STRING"},             # 5
    {"name": "description", "type": "STRING"},     # 6
    {"name": "trans_type", "type": "STRING"},      # 7
    {"name": "trans_subtype", "type": "STRING"},   # 8
    {"name": "trans_status", "type": "STRING"},    # 9
    {"name": "department", "type": "FLOAT"},       # 10
    {"name": "quantity", "type": "FLOAT"},         # 11
    {"name": "Scale", "type": "FLOAT"},            # 12
    {"name": "cost", "type": "FLOAT"},             # 13
    {"name": "unitPrice", "type": "FLOAT"},        # 14
    {"name": "total", "type": "FLOAT"},            # 15
    {"name": "regPrice", "type": "FLOAT"},         # 16
    {"name": "altPrice", "type": "FLOAT"},         # 17
    {"name": "tax", "type": "FLOAT"},              # 18
    {"name": "taxexempt", "type": "FLOAT"},        # 19
    {"name": "foodstamp", "type": "FLOAT"},        # 20
    {"name": "wicable", "type": "FLOAT"},          # 21
    {"name": "discount", "type": "FLOAT"},         # 22
    {"name": "memDiscount", "type": "FLOAT"},      # 23
    {"name": "discountable", "type": "FLOAT"},     # 24
    {"name": "discounttype", "type": "FLOAT"},     # 25
    {"name": "voided", "type": "FLOAT"},           # 26
    {"name": "percentDiscount", "type": "FLOAT"},  # 27
    {"name": "ItemQtty", "type": "FLOAT"},         # 28
    {"name": "volDiscType", "type": "FLOAT"},      # 29
    {"name": "volume", "type": "FLOAT"},           # 30
    {"name": "VolSpecial", "type": "FLOAT"},       # 31
    {"name": "mixMatch", "type": "FLOAT"},         # 32
    {"name": "matched", "type": "FLOAT"},          # 33
    {"name": "memType", "type": "BOOLEAN"},        # 34
    {"name": "staff", "type": "BOOLEAN"},          # 35
    {"name": "numflag", "type": "FLOAT"},          # 36
    {"name": "itemstatus", "type": "FLOAT"},       # 37
    {"name": "tenderstatus", "type": "FLOAT"},     # 38
    {"name": "charflag", "type": "STRING"},        # 39
    {"name": "varflag", "type": "FLOAT"},          # 40
    {"name": "batchHeaderID", "type": "BOOLEAN"},  # 41
    {"name": "local", "type": "FLOAT"},            # 42
    {"name": "organic", "type": "FLOAT"},          # 43
    {"name": "display", "type": "BOOLEAN"},        # 44
    {"name": "receipt", "type": "FLOAT"},          # 45
    {"name": "card_no", "type": "FLOAT"},          # 46
    {"name": "store", "type": "FLOAT"},            # 47
    {"name": "branch", "type": "FLOAT"},           # 48
    {"name": "match_id", "type": "FLOAT"},         # 49
    {"name": "trans_id", "type": "FLOAT"}          # 50
]

date_columns = ["datetime"]

In [27]:
folder_path = 'with_headers'

csv_files = [file for file in os.listdir (folder_path) if file.endswith('.csv')]

In [ ]:
output_folder = 'wedge_extracted'
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(folder_path):
    if filename.endswith('.zip'):
        zip_file_path = os.path.join(folder_path, filename)

        with zipfile.ZipFile(zip_file_path, 'r') as zip_file:
            # Get a list of extracted files
            extracted_files = zip_file.namelist()
            
            # Extract all files
            zip_file.extractall(output_folder)

            if len(extracted_files) == 1 and extracted_files[0].endswith('.csv'):
                csv_file_path = os.path.join(output_folder, extracted_files[0])
                new_csv_name = os.path.splitext(filename)[0] + '.csv'
                os.rename(csv_file_path, os.path.join(output_folder, new_csv_name))

print("CSV files extracted and saved in the 'wedge_extracted' folder.")


CSV files extracted and saved in the 'wedge_extracted' folder.


In [23]:
folder_path = 'wedge_extracted'
csv_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

custom_na_values = [r"\\N", r"\N", ",", ", ", "nan"] # look over the ,'s 
 
df = pd.DataFrame()

for file in csv_files:
    table_id = os.path.basename(file).split('.')[0]

    table_ref = f"{gbq_proj_id}.{gbq_dataset_id}.{table_id}"

    current_df = pd.read_csv(file, parse_dates=date_columns, na_values=custom_na_values, keep_default_na=True)


    column_types = {
    "datetime": "TIMESTAMP",  # 1
    "register_no": "float64",   # 2
    "emp_no": "float64",        # 3
    "trans_no": "float64",      # 4
    "upc": "str",          # 5
    "description": "str",  # 6
    "trans_type": "str",   # 7
    "trans_subtype": "str", # 8
    "trans_status": "str",  # 9
    "department": "float64",     # 10
    "quantity": "float64",       # 11
    "Scale": "float64",          # 12
    "cost": "float64",           # 13
    "unitPrice": "float64",      # 14
    "total": "float64",          # 15
    "regPrice": "float64",       # 16
    "altPrice": "float64",       # 17
    "tax": "float64",            # 18
    "taxexempt": "float64",      # 19
    "foodstamp": "float64",      # 20
    "wicable": "float64",        # 21
    "discount": "float64",       # 22
    "memDiscount": "float64",    # 23
    "discountable": "float64",   # 24
    "discounttype": "float64",   # 25
    "voided": "float64",         # 26
    "percentDiscount": "float64",# 27
    "ItemQtty": "float64",       # 28
    "volDiscType": "float64",    # 29
    "volume": "float64",         # 30
    "VolSpecial": "float64",     # 31
    "mixMatch": "float64",       # 32
    "matched": "float64",        # 33
    "memType": "bool",      # 34
    "staff": "bool",        # 35
    "numflag": "float64",        # 36
    "itemstatus": "float64",     # 37
    "tenderstatus": "float64",   # 38
    "charflag": "str",      # 39
    "varflag": "float64",        # 40
    "batchHeaderID": "bool", # 41
    "local": "float64",          # 42
    "organic": "float64",        # 43
    "display": "bool",      # 44
    "receipt": "float64",        # 45
    "card_no": "float64",        # 46
    "store": "float64",          # 47
    "branch": "float64",         # 48
    "match_id": "float64",       # 49
    "trans_id": "float64"        # 50
}

    for column, data_type in column_types.items():
        if data_type == "TIMESTAMP":
            current_df[column] = pd.to_datetime(current_df[column], errors='coerce')
        else:
            current_df[column] = current_df[column].astype(data_type)


    # Append the current DataFrame to the main DataFrame
    df = pd.concat([df, current_df], ignore_index=True)

In [26]:

# Upload DataFrame to BigQuery
table_name = f"{gbq_proj_id}.{gbq_dataset_id}.{table_id}"
pandas_gbq.to_gbq(df, destination_table=table_name, project_id=gbq_proj_id, if_exists='replace')

# Check if the table exists
dataset_ref = client.dataset(gbq_dataset_id)
table_ref = dataset_ref.table(table_id)

table_exists = False
retry = None  # Set retry to None if you don't have a specific retry configuration
try:
    # Attempt to get the table; if it doesn't exist, NotFound exception is raised
    client.get_table(table_ref)
    table_exists = True
except NotFound:
    pass  # Table doesn't exist

# Display the result
if not table_exists:
    print(f"The table {table_ref} does not exist.")
else:
    print(f"The table {table_ref} exists.")

100%|██████████| 1/1 [00:00<00:00, 990.39it/s]


The table leafy-sunrise-403222.wedge_data.transArchive_201701 exists.
